# Játsszunk a Scikit-learn könyvtárral

## Határozzuk meg az adattudományos alkalmazásokat

http://scikit-learn.org/stable/developers/<BR>
http://scikit-learn.org/stable/faq.html<BR>

A California Housing adathalmaz egy népszerű adathalmaz, amely a Kalifornia állambeli házak mediánértékéről tartalmaz adatokat, valamint egyéb információkat a környékről. Az adathalmaz a StatLib adattárból származik, és gyakran használják viszonyítási alapként gépi tanulási feladatoknál. Az adatokat az Amerikai Népszámlálási Hivatal és más különböző források gyűjtötték össze az 1990-es kaliforniai népszámlálásból származó információk alapján. Az adathalmaz többek között olyan jellemzőket tartalmaz, mint a népesség, a mediánjövedelem, a házak mediánértéke, valamint földrajzi koordináták. Számos, ezt az adathalmazt használó gépi tanulási feladat célja a házak mediánértékének előrejelzése a rendelkezésre álló egyéb jellemzők alapján.

A California Housing adathalmazban a célérték a házak mediánértéke az egyes tömbcsoportokban, amely 14 999 dollár és 500 001 dollár között mozog.

* Az adatkészlet az 1990-es kaliforniai népszámlálásból származó 20 640 lakásárra vonatkozó megfigyelésből áll.
* Nyolc bemeneti jellemzőt tartalmaz, köztük a szélességi és hosszúsági fokot, a mediánjövedelmet, valamint a szobák és a hálószobák számát, a népességet, a háztartások számát és a lakások mediánkorát minden egyes tömbcsoportra vonatkozóan.
* A célváltozó az egyes tömbcsoportok házainak mediánértéke.
* Az adatokon előfeldolgozást végeztek, mely során eltávolították a hiányzó értékeket, és úgy skálázták a célváltozót, hogy 0,1 és 5,0 közé essen.
* Az adathalmazt általában gépi tanulási regressziós feladatokban használják, mivel jó lehetőséget nyújt a jellemzőtervezés és a regressziós modellezés gyakorlására.

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing

def kaliforniai_lakásadatok_betöltése():
    adathalmaz = fetch_california_housing()
    X = pd.DataFrame(data=adathalmaz.data,
                     columns=adathalmaz.feature_names)
    y = pd.Series(data=adathalmaz.target, name="cél")
    return X, y

X, y = kaliforniai_lakásadatok_betöltése() 
print(f"X:{X.shape} y:{y.shape}")

X:(20640, 8) y:(20640,)


In [2]:
from sklearn.preprocessing import StandardScaler

normalizáló = StandardScaler()
normalizáló.fit(X)
normalizált_X = normalizáló.transform(X)

In [3]:
from sklearn.linear_model import LinearRegression

lineáris_regresszió = LinearRegression()
lineáris_regresszió.fit(normalizált_X, y)
print(lineáris_regresszió.coef_.round(5))

[ 0.82962  0.11875 -0.26553  0.3057  -0.0045  -0.03933 -0.89989 -0.87054]


In [4]:
értékek = [[1.21315, 32., 3.31767135, 1.07731985, 898.,
           2.1424809, 37.82, -122.27]]
megf = pd.DataFrame(értékek, columns=X.columns)

normalizált_megf = normalizáló.transform(megf)

előrejel = lineáris_regresszió.predict(normalizált_megf)
érték = előrejel[0] * 100_000
print(f"A lakások becsült mediánértéke: {érték:.2f} USD")

A lakások becsült mediánértéke: 141088.56 USD


In [5]:
lineáris_regresszió.score(normalizált_X, y)

0.606232685199805

# Használjunk transzformáló függvényeket

## Kezeljünk heterogén adatokat

In [6]:
from sklearn.compose \
    import ColumnTransformer, make_column_selector
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing \
    import StandardScaler, KBinsDiscretizer
from sklearn.linear_model import LinearRegression

X, y = kaliforniai_lakásadatok_betöltése()

In [7]:
num_oszlopok = ['MedInc', 'HouseAge', 'AveRooms',
            'AveBedrms', 'Population', 'AveOccup']
koord = ['Latitude', 'Longitude']

num_transzformáló = ColumnTransformer([
    ("normalizáló", StandardScaler(), num_oszlopok)],
     remainder="drop")

koord_transzformáló = ColumnTransformer([
    ("diszkretizáló",
     KBinsDiscretizer(n_bins=20, encode="onehot-dense"),
     koord)])

In [8]:
előfeldolgozó = FeatureUnion(
    transformer_list=[("num_transzformáló",
                        num_transzformáló),
                      ("koord_transzformáló",
                        koord_transzformáló)])

In [9]:
előfeldolgozó.fit_transform(X).shape

(20640, 46)

In [10]:
előrejelzési_folyamat = Pipeline([
    ("előfeldolgozó", előfeldolgozó),
    ("modell", LinearRegression())])

In [11]:
előrejelzési_folyamat.fit(X, y)
előrejelzési_folyamat.score(X, y)

0.666746241490374

# Foglalkozzunk az időméréssel és a teljesítménnyel

## Végezzünk teljesítményértékelést a timeit használatával

In [12]:
%timeit l = [k for k in range(10**6)]

44.5 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit -n 20 -r 5 l = [k for k in range(10**6)]

43.8 ms ± 1.65 ms per loop (mean ± std. dev. of 5 runs, 20 loops each)


In [14]:
%%timeit
l = list()
for k in range(10**6):
    l.append(k)

63.9 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
import sklearn.feature_extraction.text as szvg
darabszám_vektorizáló = szvg.CountVectorizer(
    binary=True, max_features=20)

szövegek = ["Python adattudományos célokra",
         "Python gépi tanuláshoz",
         "Mesterséges intelligencia a Pythonban"]

darabszám_vektorizáló.fit(szövegek)
vektorizált = darabszám_vektorizáló.transform(szövegek)

In [16]:
%timeit darabszám_vektorizáló.fit(szövegek)

271 µs ± 8.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [17]:
%timeit vektorizált = darabszám_vektorizáló.transform(szövegek)

57.6 µs ± 2.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
import timeit

összesített_idő = timeit.timeit(
     "vektorizált = darabszám_vektorizáló.transform(szövegek)",
     setup="from __main__ import darabszám_vektorizáló, szövegek",
     number=10000)
print(összesített_idő / 10000.0)

5.7492120000097205e-05


## Használjuk a memóriaelemzőt

In [19]:
# Telepítési eljárások
import sys
!{sys.executable} -m pip install memory_profiler

  Using cached memory_profiler-0.61.0-py3-none-any.whl (31 kB)


In [20]:
# Inicializálás az IPythonból (hogy minden IPython-indításnál ismétlődjön)
%load_ext memory_profiler

In [21]:
vektorizált = darabszám_vektorizáló.transform(szövegek)
%memit tömör_kivonatolás = vektorizált.toarray()

peak memory: 160.76 MiB, increment: 0.06 MiB


In [22]:
%%writefile példakód.py

import sklearn.feature_extraction.text as szvg

def összehasonlító_teszt(szöveg):    
    darabszám_vektorizáló = szvg.CountVectorizer(
        binary=True, max_features=20)
    darabszám_vektorizáló.fit(szöveg)
    vektorizált = darabszám_vektorizáló.transform(szöveg)
    return vektorizált.toarray()

Overwriting példakód.py


In [23]:
from példakód import összehasonlító_teszt

szövegek = ["Python adattudományos célokra",
         "Python gépi tanuláshoz",
         "Mesterséges intelligencia a Pythonban"]

%mprun -f összehasonlító_teszt összehasonlító_teszt(szövegek)

# Futtassunk kódot párhuzamosan több magon

## Szemléltessük a párhuzamos feldolgozást

In [24]:
from sklearn.datasets import load_digits
számjegyek = load_digits()

X, y = számjegyek.data, számjegyek.target
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [25]:
%timeit egy_mag = cross_val_score( \
    SVC(), X, y, cv=20, n_jobs=1)

893 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit több_mag = cross_val_score( \
    SVC(),X, y, cv=20, n_jobs=-1)

248 ms ± 3.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%timeit több_mag = cross_val_score( \
    SVC(), X, y, cv=20, n_jobs=-2)

268 ms ± 14.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
